# Un Agente RAG con LangChain 1.0 que puede hablar con un documento PDF... correctamente

## ¿Qué es RAG (Generación Aumentada por Recuperación)?

**RAG** combina dos técnicas poderosas:
1. **Recuperación**: Encontrar información relevante usando búsqueda semántica (como hicimos en `020-pdf-agent.ipynb`)
2. **Generación**: Usar un LLM para leer esa información y generar una respuesta coherente en lenguaje natural

**¿Por qué este código es un ejemplo de una aplicación RAG?**
- **Recupera** fragmentos de documento relevantes usando búsqueda semántica (a través de la herramienta)
- **Aumenta** el conocimiento del agente con ese contexto recuperado
- **Genera** una respuesta en lenguaje natural leyendo y entendiendo el texto recuperado

Esto es diferente de la simple búsqueda semántica, que sólo te da fragmentos en bruto sin interpretación.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("gen-ai-in-2026.pdf")

data = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

all_splits = text_splitter.split_documents(data)

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

ids = vector_store.add_documents(documents=all_splits)

In [ ]:
from langchain.tools import tool

@tool
def search_handbook(query: str) -> str:
    """Buscar información en el manual"""
    results = vector_store.similarity_search(query)
    return results[0].page_content

from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_handbook],
    system_prompt="Eres un agente útil que puede buscar información en el archivo PDF."
    )

from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="Según Gartner, ¿qué porcentaje de empresas usarán APIs de IA Generativa o desplegarán aplicaciones habilitadas con IA generativa en entornos de producción en 2026?")]}
)

print(response["messages"][-1].content)

## Expliquemos el código anterior en términos simples

#### Paso 1: Preparar los Datos (Igual que Antes)

Esta parte es idéntica a `020-pdf-agent.ipynb` - estamos preparando nuestra base de conocimiento:

```python
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("gen-ai-in-2026.pdf")
```
**Cargar el archivo PDF** - Apunta al documento que queremos consultar.

```python
data = loader.load()
```
**Extraer todas las páginas** - Convierte el PDF en objetos Document.

```python
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
```
**Configurar el divisor de texto** - Dividirá el documento en fragmentos de 1000 caracteres con 200 caracteres de solapamiento.

```python
all_splits = text_splitter.split_documents(data)
```
**Dividir los documentos** - Crea fragmentos más pequeños y buscables.

```python
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
```
**Crear modelo de embeddings** - Esto convertirá el texto en vectores numéricos que capturan el significado semántico.

```python
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
```
**Crear base de datos vectorial** - Una base de datos especializada para almacenar y buscar embeddings vectoriales.

```python
ids = vector_store.add_documents(documents=all_splits)
```
**Almacenar todos los fragmentos** - Cada fragmento se convierte en un vector y se almacena para su posterior recuperación.

### Paso 2: Crear el Agente RAG (¡La Parte Mágica!)

Aquí es donde ocurre RAG - le damos a un agente de IA la capacidad de buscar en el documento y generar respuestas inteligentes.

---

#### Explicación Línea por Línea

```python
from langchain.tools import tool
```
**Importar el decorador tool** - Esto nos permite crear "herramientas" que el agente puede usar. Las herramientas son funciones que el agente puede llamar cuando las necesite.

```python
@tool
def search_handbook(query: str) -> str:
    """Buscar información en el manual"""
    results = vector_store.similarity_search(query)
    return results[0].page_content
```
**Crear una herramienta de búsqueda**:
- Decorador `@tool`: Convierte esta función en una herramienta que el agente puede usar
- El **docstring** (`"""Buscar información en el manual"""`) es CRÍTICO - el agente lee esto para entender qué hace la herramienta
- `query: str`: La consulta de búsqueda (el agente la generará automáticamente basándose en la pregunta del usuario)
- `vector_store.similarity_search(query)`: Realiza búsqueda semántica para encontrar fragmentos relevantes
- `results[0].page_content`: Devuelve solo el contenido de texto del fragmento más relevante
- **Por qué funciona**: El agente puede llamar a esta función siempre que necesite información del PDF

```python
from langchain.agents import create_agent
```
**Importar el creador de agentes** - Esta es la forma principal de LangChain 1.0 para construir agentes.

```python
agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_handbook],
    system_prompt="Eres un agente útil que puede buscar información en el archivo PDF."
)
```
**Crear el agente RAG**:
- `model="gpt-4o-mini"`: El LLM que impulsará el razonamiento del agente y la generación de texto
- `tools=[search_handbook]`: Da al agente acceso a nuestra herramienta de búsqueda
- `system_prompt`: Instrucciones que le dicen al agente su propósito y capacidades
- **Lo que el agente puede hacer**: 
  - Decidir cuándo buscar en el PDF (no buscará para "Hola" o preguntas simples)
  - Generar consultas de búsqueda apropiadas
  - Leer el contenido recuperado
  - Formular respuestas en lenguaje natural
  - Gestionar preguntas de seguimiento

```python
from langchain.messages import HumanMessage
```
**Importar tipos de mensajes** - Los agentes trabajan con mensajes estructurados.

```python
response = agent.invoke(
    {"messages": [HumanMessage(content="Según Gartner, ¿qué porcentaje de empresas usarán APIs de IA Generativa o desplegarán aplicaciones habilitadas con IA generativa en entornos de producción en 2026?")]}
)
```
**Enviar una pregunta al agente**:
- `HumanMessage`: Representa un mensaje del usuario
- `agent.invoke()`: Ejecuta el agente con la conversación
- **Lo que ocurre internamente**:
  1. El agente lee la pregunta
  2. El agente decide que necesita buscar en el PDF
  3. El agente genera una buena consulta de búsqueda: `"Gartner porcentaje empresas APIs IA Generativa 2026"`
  4. El agente llama a la herramienta `search_handbook`
  5. El agente recibe el fragmento recuperado
  6. El agente lee y entiende el fragmento
  7. El agente extrae la respuesta y formula una respuesta

```python
print(response["messages"][-1].content)
```
**Imprimir la respuesta final**:
- `response["messages"]`: Contiene la conversación completa (pregunta del usuario, llamadas a herramientas, resultados de herramientas, respuesta del agente)
- `[-1]`: Obtiene el último mensaje (la respuesta final del agente)
- `.content`: Obtiene solo el texto de ese mensaje

**Salida**: `"Según Gartner, para 2026, más del 80% de las empresas usarán APIs de IA generativa o desplegarán aplicaciones habilitadas con IA generativa en entornos de producción."`

---

#### Cómo Toma Decisiones el Agente

El agente usa un **patrón ReAct** (Razonamiento + Acción):

1. **Razonamiento**: "El usuario está preguntando por una estadística de Gartner. Necesito buscar en el documento."
2. **Acción**: Llama a `search_handbook("Gartner porcentaje empresas APIs IA Generativa 2026")`
3. **Observación**: Lee el fragmento recuperado
4. **Razonamiento**: "He encontrado la respuesta en el texto: 'más del 80% de las empresas usarán APIs de IA generativa'"
5. **Acción**: Genera la respuesta final en lenguaje natural

El agente puede:
- Omitir búsquedas para preguntas simples que puede responder directamente
- Generar múltiples consultas de búsqueda si es necesario
- Refinar búsquedas si el primer resultado no es bueno
- Combinar información de múltiples fragmentos (si se configura para hacerlo)

In [ ]:
from pprint import pprint

pprint(response['messages'])

### Entendiendo el Flujo de Mensajes

La salida de `pprint(response['messages'])` muestra la conversación completa entre componentes:

1. **HumanMessage**: Tu pregunta original
2. **AIMessage #1**: El agente decidiendo usar la herramienta (contiene `tool_calls`)
3. **ToolMessage**: El resultado de `search_handbook` (el fragmento recuperado)
4. **AIMessage #2**: La respuesta final del agente después de leer el resultado de la herramienta

Esta transparencia es valiosa para depurar y entender cómo funciona el agente.

## RAG vs Búsqueda Semántica: Por Qué RAG Ofrece Mejores Respuestas

### Comparando los Dos Enfoques

| Aspecto | **Búsqueda Semántica** (020-pdf-agent.ipynb) | **Agente RAG** (021-rag-agent.ipynb) |
|---------|----------------------------------------------|--------------------------------------|
| **Qué obtienes** | Fragmentos de documento en bruto con metadatos | Respuestas en lenguaje natural |
| **¿LLM involucrado?** | No - solo similitud vectorial | Sí - el agente lee y entiende |
| **Formato de salida** | Objeto Document técnico | Texto amigable para humanos |
| **Extracción de respuestas** | Manual - tú lees el fragmento | Automática - el agente extrae la respuesta |
| **Múltiples fragmentos** | Solo muestra uno (a menos que hagas un bucle) | El agente puede sintetizar información del contexto |
| **Preguntas de seguimiento** | Sin memoria o contexto | El agente puede gestionar flujo conversacional |
| **Toma de decisiones** | Tú decides cuándo buscar | El agente decide cuándo es necesaria la recuperación |
| **Experiencia de usuario** | Pobre - requiere conocimiento técnico | Excelente - como hablar con un experto |

---

### Ejemplo de Comparación

#### Salida de Búsqueda Semántica (020-pdf-agent.ipynb):
```
page_content='Generative AI in 2026:\nTransforming Business and\n Professional Value\nAs we progress through 2026, Generative AI has reached a critical inflection point...'
metadata={'producer': 'ReportLab PDF Library', 'page': 0, 'page_label': '1', 'start_index': 0}
```
**Problemas**:
- Tienes que leer manualmente el fragmento para encontrar "80%"
- Incluye metadatos irrelevantes
- Muestra más texto del necesario
- No es conversacional ni amigable para el usuario

#### Salida del Agente RAG (021-rag-agent.ipynb):
```
Según Gartner, para 2026, más del 80% de las empresas usarán APIs de IA generativa 
o desplegarán aplicaciones habilitadas con IA generativa en entornos de producción.
```
**Beneficios**:
- Respuesta directa y clara
- Extrae exactamente lo que se preguntó
- Lenguaje natural profesional
- Listo para presentar a los usuarios

---

### Por Qué RAG Ofrece Mejores Respuestas: Las Diferencias Clave

#### 1. **Capa de Inteligencia**
- **Búsqueda Semántica**: Recuperación tonta - encuentra texto similar pero no entiende lo que necesitas
- **RAG**: Recuperación inteligente + comprensión - entiende tu pregunta, encuentra información relevante Y la interpreta

#### 2. **Respuesta vs Datos**
- **Búsqueda Semántica**: Devuelve fragmentos de datos en bruto (como dar a alguien una página entera de una enciclopedia)
- **RAG**: Devuelve respuestas precisas (como tener un experto que lee la página y te dice exactamente lo que necesitas)

#### 3. **Comprensión Contextual**
- **Búsqueda Semántica**: Sin comprensión de qué información es relevante en el fragmento
- **RAG**: El LLM lee el fragmento, lo entiende y extrae solo la información relevante

#### 4. **Razonamiento en Múltiples Pasos**
- **Búsqueda Semántica**: Una búsqueda, un resultado, hecho
- **Agente RAG**: Puede encadenar múltiples búsquedas, razonar sobre resultados y sintetizar información

#### 5. **Gestión de Complejidad**
Pregunta de ejemplo: *"Compara lo que Gartner y MIT dijeron sobre la adopción de IA"*
- **Búsqueda Semántica**: Devolvería un fragmento (estadísticas de Gartner O MIT)
- **Agente RAG**: Podría buscar estadísticas de Gartner, luego buscar estadísticas de MIT, y luego compararlas

#### 6. **Experiencia de Usuario**
- **Búsqueda Semántica**: Requiere que el usuario sea técnico, lea fragmentos, encuentre respuestas
- **Agente RAG**: Funciona como ChatGPT - conversación natural, respuestas limpias

---

### El Poder del RAG Agéntico

Esta implementación usa **RAG Agéntico**, lo que significa:

1. **Toma de Decisiones Autónoma**: El agente decide CUÁNDO buscar (no todas las consultas necesitan recuperación)
   - "Hola" → No se necesita búsqueda
   - "¿Cuánto es 2+2?" → No se necesita búsqueda  
   - "¿Qué dijo Gartner?" → Se necesita búsqueda

2. **Generación Dinámica de Consultas**: El agente crea mejores consultas de búsqueda de las que podrías hacer
   - Tu pregunta: "Según Gartner, qué porcentaje..."
   - Búsqueda del agente: "Gartner porcentaje empresas APIs IA Generativa 2026"
   - Resultado: Recuperación más enfocada

3. **Interacciones Multi-turno**: El agente puede tener conversaciones
   - Usuario: "¿Qué dijo Gartner sobre IA en 2026?"
   - Agente: *busca y responde*
   - Usuario: "¿Y qué hay de MIT?"
   - Agente: *busca de nuevo con nuevo contexto*

4. **Integración de Herramientas**: Los agentes pueden usar múltiples herramientas
   - Herramienta de búsqueda (lo que tenemos)
   - Podría añadir: búsqueda web, calculadora, consultas a bases de datos, etc.

---

### Impacto en el Mundo Real

**Por qué RAG importa en producción**:
- La investigación muestra que RAG puede mejorar la precisión de las respuestas hasta en un **70%**
- Los usuarios obtienen respuestas de calidad ChatGPT desde tus documentos privados
- No es necesario hacer fine-tuning de modelos costosos con tus datos
- Los documentos se pueden actualizar sin reentrenamiento
- Escala a millones de documentos eficientemente

**Casos de uso**:
- Bots de soporte al cliente (buscar en la base de conocimiento de la empresa)
- Análisis de documentos legales (encontrar jurisprudencia relevante)
- Revisión de literatura médica (encontrar artículos de investigación)
- Documentación empresarial (wikis internos, manuales)
- Tutores educativos (preguntas y respuestas sobre libros de texto)

---

### Conclusión Clave

**La Búsqueda Semántica** es como tener un catálogo de tarjetas de biblioteca - te dice qué libros podrían tener tu respuesta, pero aún tienes que leerlos.

**RAG** es como tener un bibliotecario investigador - encuentra los libros relevantes, los lee por ti y te da una respuesta clara y directa a tu pregunta.

**RAG Agéntico** es como tener un asistente inteligente - decide cuándo usar la biblioteca, qué buscar, e incluso puede combinar información de múltiples fuentes de forma autónoma.

---

### Camino de Evolución

```
Búsqueda por Palabras Clave → Búsqueda Semántica → RAG → RAG Agéntico → RAG Multi-Agente
         (1990s)                    (2020s)         (2023)    (2024-2026)      (Futuro)
```

Actualmente estamos en la **era del RAG Agéntico**, donde los agentes toman decisiones inteligentes sobre recuperación y razonamiento.

## Cómo ejecutar este código desde Visual Studio Code
* Abrir Terminal.
* Asegurarse de estar en la carpeta del proyecto.
* Asegurarse de tener el entorno poetry activado.
* Introducir y ejecutar el siguiente comando:
    * `python 021-rag-agent.py`